# Coding up the Strategy

Coding up the Strategy for building a trading bot as outlined by Shaun McDonogh in his [Statistical Arbitrage Bot Build course](https://www.udemy.com/course/statistical-arbitrage-bot-build-in-crypto-with-python-a-z/learn/lecture/31161552#content) on Udemy.

In [14]:
'''
    API Documentation
    https://bybit-exchange.github.io/docs/inverse/#t-introduction
'''

# API Imports
import pandas as pd
# works only with older version
# pip install pybit==1.3.6
from pybit import HTTP
from pybit import WebSocket

Coding up the Strategy for building a trading bot as outlined by Shaun McDonogh in his [Statistical Arbitrage Bot Build course](https://www.udemy.com/course/statistical-arbitrage-bot-build-in-crypto-with-python-a-z/learn/lecture/31161552#content) on Udemy.

## Configuration

In [15]:
# CONFIG
mode = "test"
# focus is going to be on hourly data (60) and daily ("D")
# available time frames
# https://bybit-exchange.github.io/docs/inverse/#tp-sl-mode-tp_sl_mode
timeframe = 60
# maximum number of history I want to pull when getting historical candlesticks
kline_limit = 200
# z_score works like a moving average
# window of this moving average needs to be configured
# the smaller the number the more aggressive the z-score strategy is going to be
z_score_window = 21

In [16]:
# LIVE API
api_key_mainnet = ""
api_secret_mainnet = ""

In [17]:
# TEST API
keys_test = pd.read_csv("~/Documents/temp/byb/testnet_key.txt", sep=" ", header=None)
api_key_testnet = keys_test[0][0]
api_secret_testnet = keys_test[0][1]

In [18]:
# SELECTED API
api_key = api_key_testnet if mode == "test" else api_key_mainnet
api_secret = api_secret_testnet if mode == "test" else api_secret_mainnet

In [19]:
# API URL
# REST endpoints, documentation at
# https://bybit-exchange.github.io/docs/inverse/#t-authentication
api_url = "https://api-testnet.bybit.com" if mode == "test" else "https://api.bybit.com"

In [20]:
# SESSION Activation
session = HTTP(api_url)

## Step (1) - Get list of symbols

In [21]:
# Get available symbols
sym_list = []
symbols = session.query_symbol()
if "ret_msg" in symbols.keys():
    if symbols["ret_msg"] == "OK":
        symbols = symbols["result"]
        for symbol in symbols:
            # (1) we only want to deal with symbols that are USDT
            # (2) we only want to trade coins that give us a rebate
            # so we're looking for a maker_fee less than zero
            # (3) only coins should be recommended that are tradable
            #if symbol["quote_currency"] == "USDT" and float(symbol["maker_fee"]) < 0 and symbol["status"] == "Trading":
            #    sym_list.append(symbol)
            # Bybit has changed policy of maker fee
            # If we filter with float(symbol["maker_fee"]) < 0 only 4 symbols will remain
            # so for now filtering without taking maker fee into account
            if symbol["quote_currency"] == "USDT" and symbol["status"] == "Trading":
                sym_list.append(symbol)

In [22]:
sym_list

[{'name': 'BTCUSDT',
  'alias': 'BTCUSDT',
  'status': 'Trading',
  'base_currency': 'BTC',
  'quote_currency': 'USDT',
  'price_scale': 2,
  'taker_fee': '0.0006',
  'maker_fee': '0.0001',
  'funding_interval': 480,
  'leverage_filter': {'min_leverage': 1,
   'max_leverage': 100,
   'leverage_step': '0.01'},
  'price_filter': {'min_price': '0.5',
   'max_price': '999999',
   'tick_size': '0.5'},
  'lot_size_filter': {'max_trading_qty': 20,
   'min_trading_qty': 0.001,
   'qty_step': 0.001,
   'post_only_max_trading_qty': '100'}},
 {'name': 'ETHUSDT',
  'alias': 'ETHUSDT',
  'status': 'Trading',
  'base_currency': 'ETH',
  'quote_currency': 'USDT',
  'price_scale': 2,
  'taker_fee': '0.0006',
  'maker_fee': '0.0001',
  'funding_interval': 480,
  'leverage_filter': {'min_leverage': 1,
   'max_leverage': 100,
   'leverage_step': '0.01'},
  'price_filter': {'min_price': '0.05',
   'max_price': '99999.9',
   'tick_size': '0.05'},
  'lot_size_filter': {'max_trading_qty': 1000,
   'min_tradi

## Step (2) - Construct and save price history